### 벅스뮤직 실시간 차트 데이터 가져오기
- 웹 크롤링 기본적인 방법으로 데이터 연결(requests)

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import sys
import pandas as pd

In [2]:
url="https://music.bugs.co.kr/chart/track/realtime/total"   #realtime, day, week
bugs_html=requests.get(url)

if bugs_html.status_code != 200:
    sys.exit('웹 페이지 정보를 겨져올 수 없습니다.')

bugs_soup=bs(bugs_html.content, "html.parser")
bugs_soup=bugs_soup.find('tbody')
tr_soup=bugs_soup.find_all('tr')    # 리스트형 구조로 "tr" 태그 전체 찾아오기

bugs_list=[]
# 순위, 곡명, 아티스트, 앨범
for tr in tr_soup:
    rank=tr.find('strong').text    # 순위 값 추출
    title=tr.find('p', class_="title").text.replace("\n", "")
    artist=tr.find('p', class_="artist").text.replace("\n", "")
    album=tr.find('a', class_="album").text.replace("\n", "")
    bugs_list.append({"순위":rank, "곡명":title, "아티스트":artist, "앨범":album})
    
df1=pd.DataFrame(bugs_list, columns=["순위", "앨범", "곡명", "아티스트"])    
df1

,순위,앨범,곡명,아티스트
0,1,Butter / Permission to Dance,Permission to Dance,방탄소년단
1,2,Weekend,Weekend,태연 (TAEYEON)
2,3,MSG워너비 1집,바라만 본다,MSG워너비(M.O.M)
3,4,Bad Habits,Bad Habits,Ed Sheeran(에드 시런)
4,5,Next Level,Next Level,aespa
...,...,...,...,...
95,96,항해,"어떻게 이별까지 사랑하겠어, 널 사랑하는 거지",AKMU (악동뮤지션)
96,97,The Prelude,At My Worst,Pink Sweat$
97,98,간 떨어지는 동거 OST Part.8,달,CHEEZE(치즈)
98,99,벌써일년,벌써일년,반하나\r반하나\r


### 멜론 차트 정보 가져오기 
- selenium 모듈 사용
- selenium 모듈은 chromedriver.exe를 이용해 크롬 브라우저를 직접 컨트롤
- 크롬 드라이버 설치: 
- 1. 구글에서 크롬드라이버 검색 또는 https://chromedriver.chromium.org/downloads 접속
- 2. 자신의 크롬브라우저 버전 확인(오른쪽 상단 점3개 클릭 => 도움말 => chrom정보)
- 3. 동일한 크롬드라이버 다운로드
- 4. 다운로드 받은 파일 위치 확인 또는 특정 폴더('c:/pydata/)에 이동

In [7]:
#!pip install selenium

In [11]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import sys
import pandas as pd
import time

In [57]:
driver=webdriver.Chrome('c:/pydata/chromedriver.exe')   # chromedriver.exe 저장 폴더. chromedriver 실행
time.sleep(2)       # 대기시간(초)

url="https://www.melon.com/chart/index.htm"   # chart, /day, /week, /month
driver.get(url)     # 실행된 chromedriver 주소창에 url 입력 후 웹사이트로 이동
driver.implicitly_wait(10)    # 사이트 화면이 나타날때까지 기다림(최대 10초)

melon_html=driver.page_source   # chromedriver 화면 전체 소스 코드 가져오기

melon_soup=bs(melon_html, "html.parser")
melon_soup=melon_soup.find('tbody')
tr_soup=melon_soup.find_all('tr')    # 리스트형 구조로 "tr" 태그 전체 찾아오기

melon_list=[]
cnt=0          # 순위 표시

for tr in tr_soup:
    cnt+=1
    rank=cnt
    title=tr.find("div", class_="ellipsis rank01").get_text().replace("\n", "")
    artist=tr.find("span", class_="checkEllipsis").get_text().replace("\n", "")
    album=tr.find("div", class_="ellipsis rank03").get_text().replace("\n", "")
    starcnt=tr.find("span", class_="cnt").get_text().replace("\n총건수\n", "").replace(",","")
    
    melon_list.append([rank,title, artist, album, starcnt])
    
df2=pd.DataFrame(melon_list, columns=['순위',"곡명","아티스트","앨범","좋아요"])
df2

,순위,곡명,아티스트,앨범,좋아요
0,1,바라만 본다,MSG워너비(M.O.M),MSG워너비 1집,107681
1,2,Next Level,aespa,Next Level,164549
2,3,Butter,방탄소년단,Butter,223986
3,4,신호등,이무진,신호등,110606
4,5,Permission to Dance,방탄소년단,Butter / Permission to Dance,116651
...,...,...,...,...,...
95,96,Don't Start Now,Dua Lipa,Future Nostalgia,142129
96,97,Love poem,아이유,Love poem,306293
97,98,내 마음이 움찔했던 순간 (취향저격 그녀 X 규현),규현 (KYUHYUN),내 마음이 움찔했던 순간 (취향저격 그녀 X 규현),106850
98,99,Hello Future,NCT DREAM,Hello Future - The 1st Album Repackage,67876


In [58]:
tr.find("div", class_="ellipsis rank01").get_text().replace("\n", "")

'밤이 되니까'

In [65]:
aa=melon_soup.select("tbody > tr")           # 태그 전체를 찾아 리스트형으로 제공, find_all()
aa[0].find("div", class_="ellipsis rank01")

<div class="ellipsis rank01"><span>
<a href="javascript:melon.play.playSong('19030101',33625988);" title="바라만 본다 재생">바라만 본다</a>
</span></div>

In [68]:
for div in melon_soup.select("div.ellipsis.rank01"):
    print(div.text.replace("\n", ""))

바라만 본다
Next Level
Butter
신호등
Permission to Dance
Weekend
헤픈 우연
치맛바람 (Chi Mat Ba Ram)
나를 아는 사람
Dun Dun Dance
Peaches (Feat. Daniel Caesar & Giveon)
롤린 (Rollin')
Alcohol-Free
Dynamite
라일락
비와 당신
ASAP
안녕 (Hello)
Celebrity
운전만해 (We Ride)
상상더하기
하루만 더
사이렌 Remix (Feat. UNEDUCATED KID, Paul Blanco)
상상더하기
Timeless
멜로디
내 손을 잡아
밝게 빛나는 별이 되어 비춰줄게
Savage Love (Laxed - Siren Beat) (BTS Remix)
밤하늘의 별을(2020)
작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey)
비 오는 날 듣기 좋은 노래 (Feat. Colde)
LOVE DAY (2021) (바른연애 길잡이 X 양요섭, 정은지)
Lovesick Girls
라라라
Dolphin
마.피.아. In the morning
잠이 오질 않네요
추억은 만남보다 이별에 남아
에잇(Prod.&Feat. SUGA of BTS)
Blueming
취기를 빌려 (취향저격 그녀 X 산들)
Life Goes On
난 너를 사랑해
봄날
어떻게 이별까지 사랑하겠어, 널 사랑하는 거지
Coin
아로하
가을 우체국 앞에서
Off My Face
다정히 내 이름을 부르면
OHAYO MY NIGHT
어푸 (Ah puh)
On The Ground
VVS (Feat. JUSTHIS) (Prod. GroovyRoom)
2002
METEOR
다시 사랑할 수 있을까
그날에 나는 맘이 편했을까
12:45 (Stripped)
내사람
러브(Prod.로코베리)
흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야
나랑 같이 걸을래 (바른연애 길잡이 X 적재)
모든 날, 모든 순간 (Every day, Every Moment)
나는 너 좋아
오래된 노래
어떻게 지

In [70]:
aa=["바라만 본다"]
aa

['바라만 본다']